# TODO: Ising Parallel Tempering with SpinSystems (MPI template)
This notebook uses `SpinSystems.IsingLatticeOptim` and compares the lowest-temperature replica against the exact Beale DOS canonical distribution.

In [ ]:
using Random
using StatsBase
using Plots
using MonteCarloX
using SpinSystems
using MPI

if !MPI.Initialized()
    MPI.Init()
end
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
nprocs = MPI.Comm_size(comm)
println("MPI rank $(rank+1)/$(nprocs) ready")

In [ ]:
L = 8
N = L * L
nrep = max(nprocs, 4)
betas = collect(range(0.20, 0.90, length=nrep))

log_dos_beale_8x8 = [
    (-128, 0.6931471805599453), (-124, 0.0), (-120, 4.852030263919617), (-116, 5.545177444479562),
    (-112, 8.449342524508063), (-108, 9.793672686528922), (-104, 11.887298863200714), (-100, 13.477180596840947),
    (-96, 15.268195474147658), (-92, 16.912371686315282), (-88, 18.59085846191256), (-84, 20.230089202801466),
    (-80, 21.870810400320693), (-76, 23.498562234123614), (-72, 25.114602234581373), (-68, 26.70699035290573),
    (-64, 28.266152815389898), (-60, 29.780704423363996), (-56, 31.241053997806176), (-52, 32.63856452513369),
    (-48, 33.96613536105969), (-44, 35.217576663643314), (-40, 36.3873411250109), (-36, 37.47007844691906),
    (-32, 38.46041522581422), (-28, 39.35282710786369), (-24, 40.141667825183845), (-20, 40.82130289691285),
    (-16, 41.38631975325592), (-12, 41.831753810069756), (-8, 42.153328313883975), (-4, 42.34770636939425),
    (0, 42.41274640460084), (4, 42.34770636939425), (8, 42.153328313883975), (12, 41.831753810069756),
    (16, 41.38631975325592), (20, 40.82130289691285), (24, 40.141667825183845), (28, 39.35282710786369),
    (32, 38.46041522581422), (36, 37.47007844691906), (40, 36.3873411250109), (44, 35.217576663643314),
    (48, 33.96613536105969), (52, 32.63856452513369), (56, 31.241053997806176), (60, 29.780704423363996),
    (64, 28.266152815389898), (68, 26.70699035290573), (72, 25.114602234581373), (76, 23.498562234123614),
    (80, 21.870810400320693), (84, 20.230089202801466), (88, 18.59085846191256), (92, 16.912371686315282),
    (96, 15.268195474147658), (100, 13.477180596840947), (104, 11.887298863200714), (108, 9.793672686528922),
    (112, 8.449342524508063), (116, 5.545177444479562), (120, 4.852030263919617), (124, 0.0), (128, 0.6931471805599453)
]

function boltzmann_pdf(beta, log_dos)
    logsum(a, b) = a > b ? a + log1p(exp(b - a)) : b + log1p(exp(a - b))
    logZ = -Inf
    for (E, logg) in log_dos
        logZ = logsum(logZ, logg - beta * E)
    end
    Dict(E => exp(logg - beta * E - logZ) for (E, logg) in log_dos)
end

In [ ]:
replicas = [Metropolis(MersenneTwister(10_000 + i); β=betas[i]) for i in 1:nrep]
systems = [IsingLatticeOptim(L, L) for _ in 1:nrep]
for i in eachindex(systems)
    init!(systems[i], :random, rng=MersenneTwister(20_000 + i))
end

pt = ParallelTempering(replicas; rng=MersenneTwister(123), rank=rank, comm_size=nprocs, local_indices=[min(rank+1, nrep)])

energy_state = Float64[energy(s) for s in systems]
for t in 1:600
    for i in eachindex(systems)
        spin_flip!(systems[i], replicas[i])
        energy_state[i] = energy(systems[i])
    end
    sweep_exchanges!(pt, energy_state; phase=iseven(t) ? 0 : 1)
end

println("Exchange attempts: ", pt.exchange_attempts, " accepted: ", pt.exchange_accepted, " rate=", round(exchange_rate(pt), digits=4))

In [ ]:
# Exact comparison for the coldest replica (largest beta)
idx_cold = argmax(betas)
beta_cold = betas[idx_cold]
sys_cold = systems[idx_cold]
alg_cold = replicas[idx_cold]

for _ in 1:(300 * N)
    spin_flip!(sys_cold, alg_cold)
end

energies = Int[]
for _ in 1:(2000 * N)
    spin_flip!(sys_cold, alg_cold)
    push!(energies, Int(energy(sys_cold)))
end

bins = collect(-2N:4:2N)
hist = fit(Histogram, energies, bins)
P_meas = normalize(hist, mode=:probability)
P_exact = boltzmann_pdf(beta_cold, log_dos_beale_8x8)
edges = collect(P_meas.edges[1])
E_mid = Int.(round.((edges[1:end-1] .+ edges[2:end]) ./ 2))
P_exact_vec = [get(P_exact, E, 0.0) for E in E_mid]

p = plot(E_mid, P_meas.weights, seriestype=:scatter, label="PT measured (cold replica)", xlabel="Energy", ylabel="Probability", title="8x8 Ising at β=$(round(beta_cold,digits=3)): measured vs exact")
plot!(p, E_mid, P_exact_vec, lw=2, color=:black, label="Exact (Beale)")
display(p)

MPI in VS Code notebooks: the kernel is usually single-process, so true multi-rank runs are usually executed from terminal with `mpiexec`. A practical workflow is notebook prototyping + equivalent `mpiexec` script for production MPI checks.